In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import itertools as it
import sympy as sp 
from sympy import symbols, Function, diff, conjugate, FiniteSet, simplify, Poly
from sympy import *
from sympy.utilities.iterables import partitions
from sympy.functions.combinatorial.numbers import nC

$n$ = dimension ($\mathbb{R}^n$ or $\mathbb{C}^n$) \
$k$ = real degree \
$(p,q)$ = complex bidegree

In [2]:
n = 3
k = 2
p = 2
q = 1
f = symbols('f', cls = Function)
x,y,z = symbols('x y z')
zs = symbols(f'z1:{n+1}', complex=True)
zbar = symbols(f'zbar1:{n+1}', complex=True)

# Newton potential (R^n)
mag_real = 0
for i in range(n):
    mag_real += zs[i]**2 ## Euclidean norm squared
np_real = sp.sqrt(mag_real)**(2-n)

# Newton potential (C^n)
mag_complex = 0
for i in range(n):
    mag_complex += zs[i]*zbar[i] ## Complex modulus squared
np_complex = sp.sqrt(mag_complex)**(2-2*n)

In [3]:
# Multi index differentiation
def D(f,alpha,zs):
    df = f
    for i in range(n):
        df = diff(df,(zs[i],alpha[i]))
    return df

In [4]:
# Returns valid n-tuples
# https://www.geeksforgeeks.org/python-program-to-convert-dictionary-to-list-by-repeating-keys-corresponding-value-times/

def get_alpha_beta(deg):
    alphas = FiniteSet()
    for part in partitions(deg, m=n):
        part = list(it.chain.from_iterable(it.repeat(k, v) for k, v in part.items()))
        while len(part) < n:
            part.append(0)
        for perm in it.permutations(part):
            alphas += FiniteSet(perm)
    return alphas

In [5]:
def make_polynomial(f, mag):
    f = sp.numer(f).subs(mag, x)
    while f.is_polynomial(zs,zbar,x) == False:
        f = simplify(f * x)
        f = sp.numer(f).subs(mag, x)
    return factor(f.subs(x,abs(z)**2))

In [6]:
def real_dim():
    return nC(n+k-1,k) - nC(n+k-3,k-2)

In [7]:
def complex_dim():
    return nC(n+p-1,p)*nC(n+q-1,q) - nC(n+p-2,p-1)*nC(n+q-2,q-1)

In [8]:
# Finding basis for H_k
def real_basis():
    basis = FiniteSet()
    alphas = get_alpha_beta(k)
    for a in alphas:
        if a[0] > 1:
            alphas -= FiniteSet(a)
    print(f'# of elements = {len(alphas)}')
    print(f'Dimension from combinatorial expression = {real_dim()} \n')
    for a in alphas:
        d = D(np_real, a, zs)
        d = make_polynomial(d, mag_real)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [9]:
# Finding basis for H_p,q
def complex_basis():
    basis = FiniteSet()
    alphas = get_alpha_beta(p)
    betas = get_alpha_beta(q)
    ab = FiniteSet()
    for a in alphas:
        for b in betas:
            if a[0]==0 or b[0]==0:
                ab += FiniteSet((a,b))
    print(f'# of elements = {len(ab)}')
    print(f'Dimension from combinatorial expression = {complex_dim()} \n')
    for (a,b) in ab:
        d = D(np_complex,b,zs)
        d = D(d,a,zbar)
        d = make_polynomial(d, mag_complex)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [10]:
real_basis()

# of elements = 5
Dimension from combinatorial expression = 5 

3*z3**2 - Abs(z)**2 

3*z2*z3 

3*z2**2 - Abs(z)**2 

3*z1*z3 

3*z1*z2 



{3*z1*z2, 3*z1*z3, 3*z2*z3, 3*z2**2 - Abs(z)**2, 3*z3**2 - Abs(z)**2}

In [11]:
complex_basis()

# of elements = 15
Dimension from combinatorial expression = 15 

-12*z3*(2*z3*zbar3 - Abs(z)**2) 

-24*z3**2*zbar2 

-24*z3**2*zbar1 

-6*z2*(4*z3*zbar3 - Abs(z)**2) 

-6*z3*(4*z2*zbar2 - Abs(z)**2) 

-24*z2*z3*zbar1 

-24*z2**2*zbar3 

-12*z2*(2*z2*zbar2 - Abs(z)**2) 

-24*z2**2*zbar1 

-6*z1*(4*z3*zbar3 - Abs(z)**2) 

-24*z1*z3*zbar2 

-24*z1*z2*zbar3 

-6*z1*(4*z2*zbar2 - Abs(z)**2) 

-24*z1**2*zbar3 

-24*z1**2*zbar2 



{-6*z1*(4*z2*zbar2 - Abs(z)**2), -6*z1*(4*z3*zbar3 - Abs(z)**2), -24*z1**2*zbar2, -24*z1**2*zbar3, -12*z2*(2*z2*zbar2 - Abs(z)**2), -6*z2*(4*z3*zbar3 - Abs(z)**2), -24*z2**2*zbar1, -24*z2**2*zbar3, -6*z3*(4*z2*zbar2 - Abs(z)**2), -12*z3*(2*z3*zbar3 - Abs(z)**2), -24*z3**2*zbar1, -24*z3**2*zbar2, -24*z1*z2*zbar3, -24*z1*z3*zbar2, -24*z2*z3*zbar1}

In [12]:
zs

(z1, z2, z3)

In [13]:
zbar

(zbar1, zbar2, zbar3)

In [14]:
np_real

1/sqrt(z1**2 + z2**2 + z3**2)

In [15]:
np_complex

(z1*zbar1 + z2*zbar2 + z3*zbar3)**(-2)